# Steam Games Dataset Analysis

This notebook analyzes the Steam Games dataset from Kaggle, which contains information about various games available on the Steam platform.

## Setup and Data Loading

In [16]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import kagglehub
import os
from pathlib import Path
import ast

# Set style for better visualizations
sns.set_style("whitegrid")  # Set seaborn style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("All libraries successfully imported and configured!")

All libraries successfully imported and configured!


In [17]:
# Define the path to save the dataset
data_dir = Path('../data')
dataset_path = data_dir / 'steam_games.csv'

# Check if the dataset already exists
if not dataset_path.exists():
    print(os.getcwd())
    print("Downloading dataset...")
    # Download the dataset
    kaggle_path = kagglehub.dataset_download("artermiloff/steam-games-dataset")
    # The downloaded path is a directory, find the CSV file
    csv_file = list(Path(kaggle_path).glob('*.csv'))[0]
    # Copy the file to our data directory
    import shutil
    shutil.copy2(csv_file, dataset_path)
    print(f"Dataset saved to {dataset_path}")
else:
    print("Dataset already exists, loading from local file...")

# Load the dataset
df = pd.read_csv(dataset_path)

# Display basic information about the dataset
print("\nDataset Shape:", df.shape)
print("\nDataset Info:")
df.info()

Dataset already exists, loading from local file...

Dataset Shape: (87806, 46)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87806 entries, 0 to 87805
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   AppID                     87806 non-null  int64  
 1   name                      87803 non-null  object 
 2   release_date              87806 non-null  object 
 3   required_age              87806 non-null  int64  
 4   price                     87806 non-null  float64
 5   dlc_count                 87806 non-null  int64  
 6   detailed_description      83656 non-null  object 
 7   about_the_game            83634 non-null  object 
 8   short_description         83713 non-null  object 
 9   reviews                   10314 non-null  object 
 10  header_image              87806 non-null  object 
 11  website                   39906 non-null  object 
 12  support_url           

## Data Cleaning and Preprocessing

In [19]:
# Drop data with missing values or insufficient data
df = df.dropna(subset=['name'])
df = df.drop(['support_email', 'support_url', 'score_rank', 'website', 'reviews', 'metacritic_url' ], 
             axis=1)

# Drop low value columns
df = df.drop(['header_image', 'short_description', 'screenshots', 'movies', 'full_audio_languages', 'average_playtime_2weeks', 'AppID',
              'median_playtime_2weeks', 'windows', 'mac', 'linux', 'packages', 'pct_pos_recent', 'num_reviews_recent',
             'positive', 'negative'], 
             axis=1)

# Combine similar columns
df['description'] = df['detailed_description'].fillna('') + ' ; ' + \
                    df['notes'].fillna('') + ' ; ' + \
                    df.apply(
                        lambda row: row['about_the_game'] if row['about_the_game'] != row['detailed_description'] else '',
                        axis=1
                    )

# Create the the union of developers and publishers
df['developers'] = df['developers'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['publishers'] = df['publishers'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['developer_publisher'] = df.apply(
    lambda row: list(set(row['developers'] + row['publishers'])) if isinstance(row['developers'], list) and isinstance(row['publishers'], list) else [],
    axis=1
)

# Function to normalize similar terms
def normalize_terms(items):
    if isinstance(items, list):
        return [item.replace('Multi-player', 'Multiplayer').replace('Single-player', 'Singleplayer') for item in items]
    return items

# Normalize 'categories' and 'genres'
df['categories'] = df['categories'].apply(normalize_terms)
df['genres'] = df['genres'].apply(normalize_terms)

# Create the 'game_classification' column as a list
df['categories'] = df['categories'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['tags'] = df['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['game_classification'] = df.apply(
    lambda row: list(set(row['categories'] + row['genres'] + list(row['tags'].keys())))
    if isinstance(row['categories'], list) and isinstance(row['genres'], list) and isinstance(row['tags'], dict)
    else [],
    axis=1
)

# Drop original columns that were combined
df  = df.drop(['detailed_description', 'about_the_game', 'notes', 'developers', 'publishers', 'categories',
              'genres', 'tags'], axis=1)

# Check for missing values
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values[missing_values > 0])

# Display basic information about the dataset
print("\nCleaned dataset shape:", df.shape)
print("\nCleaned dataset info:")
df.info()

KeyError: "['support_email', 'support_url', 'score_rank', 'website', 'reviews', 'metacritic_url'] not found in axis"

## View and Save the subset of data

In [21]:
display(df.head())

# Save the first 50 rows to a new CSV file
subset_path = data_dir / 'steam_games_subset.csv'
df_subset = df.head(50)
df_subset.to_csv(subset_path, index=False)
print(f"Subset saved to {subset_path}")

,name,release_date,required_age,price,dlc_count,metacritic_score,achievements,recommendations,supported_languages,user_score,estimated_owners,average_playtime_forever,median_playtime_forever,peak_ccu,pct_pos_total,num_reviews_total,description,developer_publisher,game_classification
0,Counter-Strike 2,2012-08-21,0,0.00,1,0,1,4111974,"['Czech', 'Danish', 'Dutch', 'English', 'Finni...",0,100000000 - 200000000,29018,6078,1362469,87,8071426,"For over two decades, Counter-Strike has offer...",[Valve],"[Online Co-Op, Military, Tactical, Trading, St..."
1,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,0,37,1701431,"['English', 'Korean', 'Simplified Chinese', 'F...",0,50000000 - 100000000,0,0,590582,58,2365012,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","[KRAFTON, Inc.]","[Adventure, Online Co-Op, Tactical, Third-Pers..."
2,Dota 2,2013-07-09,0,0.00,2,90,0,14329,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...",0,200000000 - 500000000,35557,1163,668192,81,2247365,"The most-played game on Steam. Every day, mill...",[Valve],"[SteamVR Collectibles, Online Co-Op, Character..."
3,Grand Theft Auto V,2015-04-13,17,0.00,1,96,77,1641404,"['English', 'French', 'Italian', 'German', 'Sp...",0,50000000 - 100000000,12898,6910,133571,87,1643791,"When a young street hustler, a retired bank ro...","[Rockstar North, Rockstar Games]","[Adventure, Atmospheric, Third-Person Shooter,..."
4,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,19.99,9,0,0,1090810,"['English', 'French', 'Italian', 'German', 'Sp...",0,0 - 20000,0,0,0,85,1092266,Edition Comparison Ultimate Edition The Tom Cl...,"[Ubisoft Montreal, Ubisoft]","[Atmospheric, Online Co-Op, Tactical, Online P..."
